In [8]:
import geopandas as gpd
import pandas as pd
import numpy as np
routes_df = pd.read_csv('routes.csv')
distances_df  = pd.read_csv('distances.csv')
ports_df = pd.read_csv('ports.csv')

In [5]:
ports_df["coords"] = ports_df["coords"].apply(lambda x: eval(x)[0])
ports_df["x"] = ports_df["coords"].apply(lambda x: x[0])
ports_df["y"] = ports_df["coords"].apply(lambda x: x[1])

In [6]:
df1 = distances_df.merge(ports_df, left_on='prev_port', right_on='Unnamed: 0')
df1 = df1.drop(columns = ["Unnamed: 0_y", "Unnamed: 0_x","INDEX_NO","frequency","trip_count"])
df1 = df1.rename(columns={"PORT_NAME":"prev_port_name"})
df = df1.merge(ports_df, left_on='next_port', right_on='Unnamed: 0')
df = df.drop(columns = ["Unnamed: 0", "INDEX_NO", "coords_x", "coords_y"])
df = df.rename(columns={"PORT_NAME":"next_port_name", "x_x":"x_prev", "y_x":"y_prev", "x_y":"x_next", "y_y":"y_next"})

In [9]:
ports_gdf = gpd.GeoDataFrame(
    ports_df, geometry=gpd.points_from_xy(ports_df.x,ports_df.y))

AttributeError: 'DataFrame' object has no attribute 'x'

In [10]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='cyan', edgecolor='blue')
ports_gdf.plot(ax=base, marker='o', color='red', markersize=5);


NameError: name 'ports_gdf' is not defined

In [11]:
import math

class Haversine:
    '''
    use the haversine class to calculate the distance between
    two lon/lat coordnate pairs.
    output distance available in kilometers, meters, miles, and feet.
    example usage: Haversine([lon1,lat1],[lon2,lat2]).feet
    
    '''
    def __init__(self,coord1,coord2):
        lon1,lat1=coord1
        lon2,lat2=coord2
        
        R=6371000                               # radius of Earth in meters
        phi_1=math.radians(lat1)
        phi_2=math.radians(lat2)

        delta_phi=math.radians(lat2-lat1)
        delta_lambda=math.radians(lon2-lon1)

        a=math.sin(delta_phi/2.0)**2+\
           math.cos(phi_1)*math.cos(phi_2)*\
           math.sin(delta_lambda/2.0)**2
        c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
        
        self.meters=R*c                         # output distance in meters
        self.km=self.meters/1000.0              # output distance in kilometers
        self.miles=self.meters*0.000621371      # output distance in miles
        self.feet=self.miles*5280               # output distance in feet

if __name__ == "__Haversine__":
    main()

In [12]:
chokepoints = pd.DataFrame({'Chokepoint': ["suez", "gibraltar", "malacca", "dover", "hormuz", "panama" ],
                            'Latitude': [29.957175, 35.982003, 3.142137, 51.027659, 26.669853, 8.969779], 
                            'Longitude':[32.583561,-5.452054, 100.548439, 1.483395, 56.509156, -79.580526],
                            'dist_threshold': [20, 15, 45, 40, 42, 20]}).set_index("Chokepoint") #create a dataframe with the chokepoint, lat/lon of a selected point in that chokepoint and a distance threshold (in km)

In [14]:
def route_detect(routes, chokepoints):
    """
    Detect whether there is a point on the route that goes close to the Chokepoint. 
    Returns the original route dataframe with distance to the chokepoint and whether it is affected by that chokepoint closure.
    """

    for chokepoint in chokepoints.index:
        routes["dist_to_{}".format(chokepoint)] = routes.apply(lambda x: Haversine([x.lon,x.lat], [chokepoints.loc[chokepoint]["Longitude"], chokepoints.loc[chokepoint]["Latitude"]]).km, axis=1)
        routes["affected_by_{}".format(chokepoint)] = np.where(routes["dist_to_{}".format(chokepoint)] <= chokepoints.loc[chokepoint]["dist_threshold"], True, False)

    return routes

In [13]:
r = route_detect(routes_df, chokepoints)

NameError: name 'route_detect' is not defined

In [ ]:
df = r.groupby(["prev_port", "next_port"], as_index=False).agg({'affected_by_gibraltar':'max',
                                                                'affected_by_suez':'max', 
                                                                'affected_by_malacca':'max',
                                                                'affected_by_dover':'max',
                                                                'affected_by_hormuz':'max',
                                                                'affected_by_panama':'max'}) # we groupby the individual route 
df.to_csv("route_blocks")
